`python3 -m pip install -U pandas plotly nbformat networkx`

`pip install "https://github.com/DCMLab/wavescapes/archive/refs/heads/johannes.zip"`

In [1]:
%reload_ext autoreload
%autoreload 2
import numpy as np
from wavescapes import Wavescape, circular_hue

from etl import get_dfts, get_human_analyses, get_mag_phase_mx, get_metadata, get_most_resonant, get_pcms, get_pcvs, test_dict_keys, \
  get_correlations, make_feature_vectors
from utils import get_coeff, most_resonant2color, long2utm

## Settings

In [2]:
DEBUSSY_REPO = '.'
DATA_FOLDER = '~/DATA/debussy_figures/data'
#DATA_FOLDER = './data'
EXAMPLE_FNAME = 'l123-08_preludes_ondine'
LONG_FORMAT = True

## Loading metadata
Metadata for all pieces contained in the dataset.

In [3]:
metadata = get_metadata(DEBUSSY_REPO)
metadata.columns

Metadata for 82 files.


Index(['rel_paths', 'last_mc', 'last_mn', 'length_qb', 'length_qb_unfolded',
       'all_notes_qb', 'n_onsets', 'TimeSig', 'KeySig', 'label_count',
       'composer', 'workTitle', 'movementNumber', 'movementTitle',
       'workNumber', 'poet', 'lyricist', 'arranger', 'copyright',
       'creationDate', 'mscVersion', 'platform', 'source', 'translator',
       'musescore', 'ambitus', 'comment', 'comments', 'composed_end',
       'composed_start', 'originalFormat', 'pdf', 'staff_1_ambitus',
       'staff_1_instrument', 'staff_2_ambitus', 'staff_2_instrument',
       'staff_3_ambitus', 'staff_3_instrument', 'transcriber', 'typesetter',
       'year', 'median_recording', 'qb_per_minute', 'sounding_notes_per_qb',
       'sounding_notes_per_minute'],
      dtype='object')

The column `year` contains composition years as the middle between beginning and end  of the composition span.

In [4]:
metadata.year.head(10)

fnames
l000_etude                     1915.0
l000_soirs                     1917.0
l009_danse                     1880.0
l066-01_arabesques_premiere    1888.0
l066-02_arabesques_deuxieme    1891.0
l067_mazurka                   1890.0
l068_reverie                   1890.0
l069_tarentelle                1890.0
l070_ballade                   1890.0
l071_valse                     1890.0
Name: year, dtype: float64

Series `median_recording` contains median recording times in seconds, retrieved from the Spotify API. the Spotify API.

In [5]:
metadata.median_recording.head(10)

fnames
l000_etude                     272.5530
l000_soirs                     145.8265
l009_danse                     124.5995
l066-01_arabesques_premiere    240.7780
l066-02_arabesques_deuxieme    213.9330
l067_mazurka                   175.9130
l068_reverie                   265.4265
l069_tarentelle                331.5290
l070_ballade                   396.0200
l071_valse                     221.4500
Name: median_recording, dtype: float64

Columns mirroring a piece's activity are currently:
* `qb_per_minute`: the pieces' lengths (expressed as 'qb' = quarterbeats) normalized by the median recording times; a proxy for the tempo
* `sounding_notes_per_minute`: the summed length of all notes normalized by the piece's duration (in minutes)
* `sounding_notes_per_qb`: the summed length of all notes normalized by the piece's length (in qb)
Other measures of activity could be, for example, 'onsets per beat/second' or 'distinct pitch classes per beat/second'.

## Loading human analysis data

In [6]:
analyses = get_human_analyses(DEBUSSY_REPO)
analyses.head()

,L,mc_start,mc_start_offset,mc_end,mc_end_offset,structure,transposition,comments,source,fname,from_qb,to_qb
0,70,4.0,0,6.0,0,octa,"1,2",NaN,Forte,l070_ballade,12.0,20.0
1,70,35.0,0,37.0,0,octa,"1,2",Eb odd note,Forte,l070_ballade,136.0,144.0
2,106,7.0,0,10.0,0,acoustic,A acoustic,NaN,Tymoczko,l106_isle,24.0,36.0
3,106,15.0,0,21.0,0,acoustic,A acoustic,NaN,Tymoczko,l106_isle,56.0,80.0
4,106,64.0,0,67.0,0,acoustic,A acoustic,NaN,Tymoczko,l106_isle,162.0,174.125


## Loading Pitch Class Vectors (PCVs)
An `{fname -> pd.DataFrame}` dictionary where each `(NX12)` DataFrame contains the absolute durations (expressed in quarter nots) of the 12 chromatic pitch classes for the `N` slices of length = 1 quarter note that make up the piece `fname`. The IntervalIndex reflects each slice's position in the piece. Set `pandas` to False to retrieve NumPy arrays without the IntervalIndex and column names.

In [7]:
pcvs = get_pcvs(DEBUSSY_REPO, pandas=True)
test_dict_keys(pcvs, metadata)
pcvs[EXAMPLE_FNAME].head(5)

Found matrices for all files listed in metadata.tsv.


,0,1,2,3,4,5,6,7,8,9,10,11
iv,,,,,,,,,,,,
"[0.0, 1.0)",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.5,0.5,0.0
"[1.0, 2.0)",0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
"[2.0, 3.0)",0.0,1.5,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.5,0.5,0.0
"[3.0, 4.0)",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.5,0.0,0.5
"[4.0, 5.0)",0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


## Loading Pitch Class Matrices
An `{fname -> np.array}` dictionary where each `(NxNx12)` array contains the aggregated PCVs for all segments that make up a piece. The square matrices contain values only in the upper right triangle, with the lower left beneath the diagonal is filled with zeros. The values are arranged such that row 0 correponds to the original PCV, row 1 the aggregated PCVs for all segments of length = 2 quarter notes, etc. For getting the segment reaching from slice 3 to 5 (including), i.e. length 3, the coordinates are `(2, 5)` (think x = 'length - 1' and y = index of the last slice included). The following example shows the upper left 3x3 submatrix where the first three entries (which are PCVs of size 12) correspond to the first three PCVs above, the first three of the second row to their sums padded with a 0-PCV, and the first three of the third row corresponding to the sum of row 0, padded with another 0-PCV.

In [8]:
pcms = get_pcms(DEBUSSY_REPO, long=LONG_FORMAT)
test_dict_keys(pcms, metadata)
pcms[EXAMPLE_FNAME].shape

Found matrices for all files listed in metadata.tsv.


(24976, 12)

## Loading Discrete Fourier Transforms
`{fname -> np.array}` containing `(NxNx7)` complex matrices. For instance, here's the first element, a size 7 complex vector with DFT coefficients 0 through 6:

In [9]:
dfts = get_dfts(DEBUSSY_REPO, long=LONG_FORMAT)
test_dict_keys(dfts, metadata)
dfts[EXAMPLE_FNAME].shape

Found matrices for all files listed in metadata.tsv.


(24976, 7)

You can view the 7 complex numbers as magnitude-phase pairs

In [10]:
get_coeff(dfts[EXAMPLE_FNAME], 0, 0)

array([[ 2.5 ,  0.  ],
       [ 0.97,  0.79],
       [ 0.87, -1.57],
       [ 1.12, -2.03],
       [ 1.8 , -1.85],
       [ 0.26, -2.36],
       [ 1.5 ,  3.14]])

or even as strings where the numbers are rounded and angles are shown in degrees:

In [11]:
get_coeff(dfts[EXAMPLE_FNAME], 0, 0, deg=True)

array([['2.5+0°'],
       ['0.97+315°'],
       ['0.87+90°'],
       ['1.12+117°'],
       ['1.8+106°'],
       ['0.26+135°'],
       ['1.5+180°']], dtype='<U9')

## Loading magnitude-phase matrices
`{fname -> np.array}` where each of the `(NxNx6x2)` matrices contains the 6 relevant DFT coefficients converted into magnitude-phase pairs where the magnitudes have undergone at least one normalization, i.e. are all within [0,1]. The files have been pre-computed and are loaded from g-zipped pickled matrices.

The parameter `norm_params` can be one or several `(how, indulge)` pairs where `indulge` is a boolean and `how ∈ {'0c', 'post_norm', 'max_weighted', 'max'}`.

In [12]:
norm_params = ('0c', True)
mag_phase_mx_dict = get_mag_phase_mx(DATA_FOLDER, norm_params=norm_params, long=LONG_FORMAT)
test_dict_keys(mag_phase_mx_dict, metadata)
mag_phase_mx_dict[EXAMPLE_FNAME].shape


Found matrices for all files listed in metadata.tsv.


(24976, 6, 2)

In [13]:
mpm = mag_phase_mx_dict[EXAMPLE_FNAME]
colors = circular_hue(mpm[...,1,:], output_rgba=True, ignore_phase=True)
colors.shape

(24976, 1)

In [14]:
circular_hue(long2utm(mpm[..., 1, :])).shape

(223, 223)

In [15]:
long2utm(colors)[...,0].shape

(223, 223)

Note that the phases (2nd column) are the same that we inspected above via `get_coeff()` whereas the magnitudes are now normalized by the first (now absent) coefficient 0.

In [16]:
mag_phase_mx_dict[EXAMPLE_FNAME][0]

array([[ 0.4       ,  0.78539816],
       [ 0.4       , -1.57079633],
       [ 0.6       , -2.03444394],
       [ 1.        , -1.85183123],
       [ 0.10717968, -2.35619449],
       [ 0.6       ,  3.14159265]])

## Loading most resonant DFT coefficients
This cell depends on the previously loaded magnitude-phase matrices, i.e. a conscious choice of a normalization method has been made above.

`get_most_resonant` returns three `{fname -> nd.array}` dictionaries where for each piece, the three `(NxN)` matrices correspond to

1. the index between 0 and 5 of the most resonant of the six DFT coefficient 1 through 6
2. its magnitude
3. the inverse entropy of the 6 magnitudes

In [17]:
max_coeffs, max_mags, inv_entropies = get_most_resonant(mag_phase_mx_dict, )
np.column_stack((max_coeffs[EXAMPLE_FNAME][:3],
max_mags[EXAMPLE_FNAME][:3],
inv_entropies[EXAMPLE_FNAME][:3]))

/home/hentsche/miniconda3/lib/python3.9/site-packages/scipy/stats/_entropy.py:77: RuntimeWarning: invalid value encountered in true_divide
  pk = 1.0*pk / np.sum(pk, axis=axis, keepdims=True)


array([[3.        , 1.        , 0.21324779],
       [2.        , 1.        , 0.04483897],
       [3.        , 1.        , 0.11747646]])

## Loading major, minor, and tritone correlations

This cell loads pickled matrices. To re-compute correlations from pitch-class matrices, use `get_maj_min_coeffs()` for major and minor correlations and `get_ttms()` for tritone-ness matrices.

In [18]:
correl_dict = get_correlations(DATA_FOLDER, long=LONG_FORMAT)
test_dict_keys(correl_dict, metadata)
correl_dict[EXAMPLE_FNAME].shape

Found matrices for all files listed in metadata.tsv.


(24976, 3)

## Loading pickled 9-fold vectors

The function is a shortcut for
* loading a particular kind of pickled normalized magnitude-phase-matrices
* loading pickled tritone, major, and minor coefficients
* concatenating them toegther

In [19]:
norm_params = ('0c', True)
ninefold_dict = make_feature_vectors(DATA_FOLDER, norm_params=norm_params, long=LONG_FORMAT)
test_dict_keys(ninefold_dict, metadata)
ninefold_dict[EXAMPLE_FNAME].shape

Found matrices for all files listed in metadata.tsv.


(24976, 9)

## Minimal example for drawing a wavescape

In [20]:
#metadata.index.unique()

In [21]:
examples_ambiguous = ['l123-08_preludes_ondine', 'l110-01_images_reflets']
examples_plain = ['l075-03_suite_clair', 'l066-01_arabesques_premiere']

In [25]:
import matplotlib.pyplot as plt

for EXAMPLE_FNAME in examples_ambiguous:
    wv_cols = circular_hue(mag_phase_mx_dict[EXAMPLE_FNAME])
    if LONG_FORMAT:
        wv_cols = long2utm(wv_cols)
    length = wv_cols.shape[0]
    width = 500 if length <= 250 else 2 * length
    print(wv_cols.shape)
    for coeff in range(6):
        ws = Wavescape(wv_cols[...,coeff], width=width)
        ws.draw()
        plt.suptitle(f"{EXAMPLE_FNAME}: c{coeff+1}", fontsize=14)
        #plt.legend(handles=patches)
        plt.savefig('./figures/' + f"{EXAMPLE_FNAME}-c{coeff+1}")
        plt.close()
        

(223, 223, 6)
(190, 190, 6)


## Drawing summary wavescapes and their associated color legend

Colour legend for most resonant coefficients:

![legend](legend.png)

In [26]:
for EXAMPLE_FNAME in examples_ambiguous+examples_plain:
    max_coeff, max_mag, inv_entropy = max_coeffs[EXAMPLE_FNAME], max_mags[EXAMPLE_FNAME], inv_entropies[EXAMPLE_FNAME]
    if LONG_FORMAT:
        max_coeff, max_mag, inv_entropy = long2utm(max_coeff), long2utm(max_mag), long2utm(inv_entropy)
    draw_settings = dict(label_size=5, aw_per_tick=10, tick_factor=10, tight_layout=False)
    length = max_coeff.shape[0]
    width = 500 if length <= 250 else 2 * length
    summary_by_max = most_resonant2color(max_coeff, max_mag)
    ws = Wavescape(summary_by_max, width=width)
    ws.draw(label=f"Summary wavescape for {EXAMPLE_FNAME}\nOpacity = magnitude of the most resonant", **draw_settings)
    plt.savefig('./figures/' + str(EXAMPLE_FNAME + '_summary_wavescape_mg') + '.png')
    plt.close()
    

In [27]:
for EXAMPLE_FNAME in examples_ambiguous+examples_plain:
    max_coeff, max_mag, inv_entropy = max_coeffs[EXAMPLE_FNAME], max_mags[EXAMPLE_FNAME], inv_entropies[EXAMPLE_FNAME]
    if LONG_FORMAT:
        max_coeff, max_mag, inv_entropy = long2utm(max_coeff), long2utm(max_mag), long2utm(inv_entropy)
    draw_settings = dict(label_size=5, aw_per_tick=10, tick_factor=10, tight_layout=False)
    length = max_coeff.shape[0]
    width = 500 if length <= 250 else 2 * length
    summary_by_ent = most_resonant2color(max_coeff, inv_entropy)
    ws = Wavescape(summary_by_ent, width=width)
    ws.draw(label=f"Summary wavescape for {EXAMPLE_FNAME}\nOpacity = inverse entropy of the 6 coeffs", **draw_settings)
    plt.savefig('./figures/' + str(EXAMPLE_FNAME + '_summary_wavescape_ent') + '.png')
    plt.close()
    